In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
%run "/eReport/PkgRcd/Cálculo de Provisiones"

In [0]:
import pandas as pd
from datetime import datetime

class PrcRcdProcesaOrigen():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.empresa = parametros['empresa']  
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d') 
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def CargaClienteSaldos(self):
    try:
      #Reproceso
      deleteClienteSaldos = """DELETE CPC
                              FROM    CLIENTE C
                                      JOIN CLIENTE_PRODUCTO CP ON C.ID = CP.CLIENTE
                                      JOIN CLIENTE_PRODUCTO_CONCEPTO CPC ON CPC.CLIENTE_PRODUCTO = CP.ID
                              WHERE   FECHA_PROCESO =  """"'"+ self.fecha_proceso_str +"""'"""
      self.cursor.execute(deleteClienteSaldos)
      self.connection.commit()
      
      #Proceso
      
      #actualizamos la clasificacion interna de cliente producto y Cliente  
      actualizaClasifInterna_cp = """SELECT B.CLASIFICACION_DEUDOR, A.ID
                                      FROM CLIENTE_PRODUCTO A
                                      JOIN CLASIFICACION_DEUDOR_DIAS_ATRASO B ON A.TIPO_CREDITO_SBS = B.TIPO_CREDITO_SBS
                                      AND A.DIAS_ATRASO BETWEEN B.DIAS_ATRASO_MINIMO AND ISNULL(B.DIAS_ATRASO_MAXIMO, DIAS_ATRASO+1)
                                      JOIN CLIENTE C ON C.ID = A.CLIENTE
                                      WHERE C.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
                                  """
      sql_actualizaClasifInterna_cp = pd.read_sql_query(actualizaClasifInterna_cp, self.connection)
      df_actualizaClasifInterna_cp = pd.DataFrame(sql_actualizaClasifInterna_cp)

      actualizaClasificacionInterna_cp = "UPDATE [DBO].CLIENTE_PRODUCTO SET CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO= ? WHERE ID =?"  
      self.cursor.fast_executemany = True
      self.cursor.executemany(actualizaClasificacionInterna_cp, df_actualizaClasifInterna_cp.values.tolist())
      self.connection.commit()    

      actualizaClasifInterna_c = """SELECT B.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO, B.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO AS CLASIFICACION_DEUDOR, B.DIAS_ATRASO, A.ID
                                      FROM CLIENTE A
                                      JOIN CLIENTE_PRODUCTO B ON A.ID = B.CLIENTE
                                      WHERE A.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
                                      """
      sql_actualizaClasifInterna_c = pd.read_sql_query(actualizaClasifInterna_c, self.connection)
      df_actualizaClasifInterna_c = pd.DataFrame(sql_actualizaClasifInterna_c)

      actualizaClasificacionInterna_c = "UPDATE [DBO].CLIENTE SET CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO= ?, CLASIFICACION_DEUDOR=?, DIAS_ATRASO=? WHERE ID =?"
      self.cursor.fast_executemany = True
      self.cursor.executemany(actualizaClasificacionInterna_c, df_actualizaClasifInterna_c.values.tolist())
      self.connection.commit()
      
      selectSaldos = """SELECT clienteProducto.ID AS CLIENTE_PRODUCTO, CASE WHEN formato.MONEDA='01' THEN 'PEN' ELSE 'USD' END AS MONEDA,
                                 saldos.SALDO_NORMAL AS CAPITAL_VIGENTE,
                                 saldos.SALDO_VENCIDO AS CAPITAL_VENCIDO,
                                 saldos.INTERES AS INTERES_VIGENTE,
                                 saldos.INTERES_VENCIDO AS INTERES_VENCIDO,
                                 saldos.COMISION,
                                 saldos.SALDO_PAGO_ADELANTADO AS PAGO_ANTICIPADO,
                                 saldos.LINEA_OTORGADA AS LINEA_CREDITO,
                                 CASE
                                     WHEN clienteProducto.SITUACION_CREDITO ='C' THEN 0
                                     WHEN saldos.SALDO_NORMAL + saldos.SALDO_VENCIDO > saldos.LINEA_OTORGADA THEN 0
                                     ELSE saldos.LINEA_OTORGADA - (saldos.SALDO_NORMAL + saldos.SALDO_VENCIDO)
                                 END AS LINEA_DISPONIBLE_CREDITO
                          FROM [BF_RCD_SALDOS] saldos
                          JOIN [BF_FORMATO_RCD] formato ON formato.NUM_TARJETA = saldos.CUENTA_CMR
                          JOIN CLIENTE_DEUDOR clienteDeudor ON clienteDeudor.CODIGO_DEUDOR_INTERNO = saldos.CUENTA_CMR
                          JOIN CLIENTE cliente ON cliente.CLIENTE_DEUDOR = clienteDeudor.ID
                          JOIN CLIENTE_PRODUCTO clienteProducto ON clienteProducto.CLIENTE = cliente.ID
                          WHERE cliente.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""' 
                          AND (saldos.LINEA_OTORGADA > 0 OR (saldos.SALDO_NORMAL + saldos.SALDO_VENCIDO) > 0)
                          """
      sql_selectSaldos = pd.read_sql_query(selectSaldos, self.connection)
      df_selectSaldos = pd.DataFrame(sql_selectSaldos)
      
      #Extraemos la moneda
      df_moneda = df_selectSaldos[['CLIENTE_PRODUCTO','MONEDA']]

      #eliminamos moneda
      df_selectSaldos.drop('MONEDA', axis = 'columns', inplace=True)
      
      #obtenemos la configuracion por producto
      productoConceptos = """SELECT E.FECHA_PROCESO,
                             F.EMPRESA,
                             B.PRODUCTO,
                             A.TIPO_CLIENTE,
                             A.TIPO_CREDITO,
                             A.TIPO_CREDITO_SBS,
                             A.SITUACION_CREDITO,
                             A.INDICADOR_PAGO_CASTIGO,
                             E.CLASIFICACION_DEUDOR,
                             E.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO,
                             E.CLASIFICACION_DEUDOR_CON_ALINEAMIENTO,
                             A.ID CLIENTE_PRODUCTO
                      FROM CLIENTE_PRODUCTO A
                      JOIN CLIENTE_DEUDOR_PRODUCTO B ON A.CLIENTE_DEUDOR_PRODUCTO = B.ID
                      JOIN CLIENTE E ON E.ID = A.CLIENTE
                      JOIN CLIENTE_DEUDOR F ON F.ID = E.CLIENTE_DEUDOR
                      JOIN TIPO_CREDITO_SBS D ON D.ID = A.TIPO_CREDITO_SBS
                      WHERE E.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
      """
      sql_productoConceptos = pd.read_sql_query(productoConceptos, self.connection)
      df_productoConceptos = pd.DataFrame(sql_productoConceptos)

      #pivoteamos las columnas de importes
      df_unpivoted_saldos = df_selectSaldos.melt(id_vars=['CLIENTE_PRODUCTO'], var_name='CONCEPTO', value_name='IMPORTE_INGRESO')
      
      df_unpivoted_saldos = df_unpivoted_saldos[df_unpivoted_saldos['IMPORTE_INGRESO'] > 0]

      #ordenamos por concepto saldo
      df_unpivoted_saldos.sort_values("CONCEPTO", inplace = True)
      
      #Agregamos concepto saldo principal
      df_unpivoted_saldos['CONCEPTO_SALDO_PRINCIPAL'] = np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('CAPITAL_VIGENTE|CAPITAL_VENCIDO|INTERES_VIGENTE|INTERES_VENCIDO|INGRESO_DIFERIDO|COMISION',regex=True),38,
                                                                 np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('LINEA_CREDITO',regex=True),21,
                                                                          np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('LINEA_DISPONIBLE_CREDITO',regex=True),16, 38)

                                                                         )
                                                                )
      #Agregamos concepto saldo
      df_unpivoted_saldos['CONCEPTO_SALDO'] = np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('CAPITAL_VIGENTE|CAPITAL_VENCIDO',regex=True),38,
                                                       np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('COMISION',regex=True),65,
                                                                np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('INGRESO_DIFERIDO',regex=True),55,
                                                                         np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('INTERES_VIGENTE|INTERES_VENCIDO',regex=True),2,
                                                                                  np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('LINEA_CREDITO',regex=True),21,
                                                                                           np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('LINEA_DISPONIBLE_CREDITO',regex=True),16,38)
                                                                                          )
                                                                                 )
                                                                        )
                                                               )
                                                      )

      #Agregamos concepto clasificacion
      df_unpivoted_saldos['CONCEPTO_CLASIFICACION'] = np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('CAPITAL_VENCIDO|INTERES_VENCIDO',regex=True), 2,
                                                               np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('CAPITAL_VIGENTE',regex=True), 1,
                                                                        np.where(df_unpivoted_saldos['CONCEPTO'].str.contains('LINEA',regex=True), 3,
                                                                                 1,
                                                                                )
                                                                       )
                                                             )
      
      df_merge1 = pd.merge(df_unpivoted_saldos,df_productoConceptos,on='CLIENTE_PRODUCTO', how='inner')
      df_merge2 = pd.merge(df_merge1,df_moneda,on='CLIENTE_PRODUCTO', how='inner')
      
      #Agregamos importe solarizado
      df_cambioDiario = PrcGeneral().cambioDiario(self.fecha_proceso_str,'CIE')
      df_merge2['IMPORTE'] = np.where(df_merge2['MONEDA'] == 'USD', df_merge2['IMPORTE_INGRESO'] * df_cambioDiario,df_merge2['IMPORTE_INGRESO'])
      df_merge2['IMPORTE'] = np.round(df_merge2['IMPORTE'], decimals=2)
      
      #Agregamos cuenta contable
      df_cuenta_contable_vigente = PRC_RCD_PROVISIONES(4).cuentaContableVigente(self.fecha_proceso_str)
      
      df_merge3 = pd.merge(df_merge2,  df_cuenta_contable_vigente, on=['EMPRESA','PRODUCTO','TIPO_CLIENTE','TIPO_CREDITO','TIPO_CREDITO_SBS','SITUACION_CREDITO','INDICADOR_PAGO_CASTIGO','CLASIFICACION_DEUDOR','CONCEPTO_SALDO_PRINCIPAL','CONCEPTO_SALDO','CONCEPTO_CLASIFICACION','MONEDA'], how='inner')

      df_merge3['ID'] = pd.to_numeric(df_merge3['ID'], errors='coerce').replace(np.nan,0)
    
      #ordenamos informacion
      df_saldos = df_merge3[['CLIENTE_PRODUCTO','CONCEPTO_SALDO_PRINCIPAL','CONCEPTO_SALDO','CONCEPTO_CLASIFICACION','MONEDA','ID','IMPORTE','IMPORTE_INGRESO']]
      #df_saldos = df_saldos.groupby(['CLIENTE_PRODUCTO','CONCEPTO_SALDO_PRINCIPAL','CONCEPTO_SALDO','CONCEPTO_CLASIFICACION','MONEDA','ID']).agg('sum')
      insert = "INSERT INTO [dbo].[CLIENTE_PRODUCTO_CONCEPTO] VALUES (?,?,?,?,?,?,?,?)"
      self.cursor.fast_executemany = True
      self.cursor.executemany(insert, df_saldos.values.tolist())
      self.connection.commit() 
      print("proceso_finalizado")
      
    except Exception as  e:
      self.connection.rollback()
      print("Error: " + str(e))

#objeto = PrcRcdProcesaOrigen(4)
#objeto.CargaClienteSaldos()
  
    